<a href="https://colab.research.google.com/github/ChristianIES/Redes_Neuronales/blob/main/RN_2022_DL_Tarea_3_Proyecto_de_Investigaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea 1 - El Perceptrón Multicapa como modelo de regresión

- Nombre: Valentina Bastidas S.
- Programa: Doctorado en Estadística


- Nombre: Christian Araya M.
- Programa: Doctorado en Estadística




Códigos asociados al proyecto semestral del curso Redes Neuronales, Optativo para el programa de Doctorado en Estadística.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
from scipy.stats import pearsonr
from scipy.stats import spearmanr

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

from keras.models import Sequential
from keras.layers import Dense

%matplotlib inline

In [2]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [3]:
datos=pd.read_csv("/content/Maternal Health Risk Data Set.csv",header=0)
datos.head()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98.0,86,high risk
1,35,140,90,13.0,98.0,70,high risk
2,29,90,70,8.0,100.0,80,high risk
3,30,140,85,7.0,98.0,70,high risk
4,35,120,60,6.1,98.0,76,low risk


In [4]:
#existencia de datos perdidos:
datos.isnull().sum()

Age            0
SystolicBP     0
DiastolicBP    0
BS             0
BodyTemp       0
HeartRate      0
RiskLevel      0
dtype: int64

In [5]:
dataset=datos.values
X=dataset[:,:-1]
y=dataset[:,-1]

In [6]:
y=y.reshape((len(y),1))
y

array([['high risk'],
       ['high risk'],
       ['high risk'],
       ...,
       ['high risk'],
       ['high risk'],
       ['mid risk']], dtype=object)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [8]:
#escalamiento de los datos:
scaler = MinMaxScaler(feature_range=(0, 1))
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [9]:
#Árboles de decisión:

from sklearn import tree

arbol = tree.DecisionTreeClassifier(criterion='gini')

#entrenamiento:
arbol.fit(X_train, y_train)

print("Train accuracy:",arbol.score(X_train,y_train))
print("Test accuracy:",arbol.score(X_test,y_test))



Train accuracy: 0.930780559646539
Test accuracy: 0.7940298507462686


In [54]:
y_pred = arbol.predict(X_test)
#print(y_pred)


In [55]:
cm1 = confusion_matrix(y_test, y_pred)
print(f'CM:',cm1)
print(f'Accuracy:',accuracy_score(y_test, y_pred)* 100 ,'%')
print(classification_report(y_test, arbol.predict(X_test)))

CM: [[ 85   0   9]
 [  9 106  24]
 [ 10  17  75]]
Accuracy: 79.40298507462687 %
              precision    recall  f1-score   support

   high risk       0.82      0.90      0.86        94
    low risk       0.86      0.76      0.81       139
    mid risk       0.69      0.74      0.71       102

    accuracy                           0.79       335
   macro avg       0.79      0.80      0.79       335
weighted avg       0.80      0.79      0.79       335



In [11]:
#Random Forest:

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

#entrenamiento:
rf.fit(X_train, y_train)

print("Train accuracy:",rf.score(X_train,y_train))
print("Test accuracy:",rf.score(X_test,y_test))

<ipython-input-11-bafa7bbfb7bd>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


Train accuracy: 0.930780559646539
Test accuracy: 0.8059701492537313


In [12]:
y_pred2 = rf.predict(X_test)

In [13]:
cm2 = confusion_matrix(y_test, y_pred2)
print(f'CM:',cm2)
print(f'Accuracy:',accuracy_score(y_test, y_pred2)* 100 ,'%')
print(classification_report(y_test, rf.predict(X_test)))

CM: [[ 83   2   9]
 [  5 109  25]
 [  8  16  78]]
Accuracy: 80.59701492537313 %
              precision    recall  f1-score   support

   high risk       0.86      0.88      0.87        94
    low risk       0.86      0.78      0.82       139
    mid risk       0.70      0.76      0.73       102

    accuracy                           0.81       335
   macro avg       0.81      0.81      0.81       335
weighted avg       0.81      0.81      0.81       335



In [14]:
#preparación de los datos para una red neuronal:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y_train)
encoder.fit(y_test)
encoded_ytrain = encoder.transform(y_train)
encoded_ytest = encoder.transform(y_test)

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
from keras.utils import np_utils
dummy_ytrain = np_utils.to_categorical(encoded_ytrain)
dummy_ytest = np_utils.to_categorical(encoded_ytest)

In [34]:
#modelo1: 1 capa de entrada, 1 capa oculta de 50 neuronas, 1 capa de salida de 3 neuronas:
model = Sequential()
model.add(Dense(50, input_dim=6, activation='relu'))
model.add(Dense(3, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 50)                350       
                                                                 
 dense_9 (Dense)             (None, 3)                 153       
                                                                 
Total params: 503
Trainable params: 503
Non-trainable params: 0
_________________________________________________________________


In [ ]:
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=1000)
mc = ModelCheckpoint('best_model.h5', monitor='accuracy', mode='max', verbose=1, save_best_only=True)
model.fit(X_train, dummy_ytrain, epochs=1000, batch_size=16, verbose=2, callbacks=[es,mc])

In [37]:
modelo = load_model('best_model.h5')
ypred3 = modelo.predict(X_test)
ypred3 = np.argmax(ypred3, axis=1)
ypred3 = encoder.inverse_transform(ypred3)

11/11 [==============================] - 0s 3ms/step


In [38]:
cm3 = confusion_matrix(y_test, ypred3)
print(f'CM:',cm3)
print(f'Accuracy:',accuracy_score(y_test, ypred3)* 100 ,'%')
print(classification_report(y_test, ypred3))

CM: [[ 79   2  13]
 [  5 112  22]
 [  9  48  45]]
Accuracy: 70.44776119402985 %
              precision    recall  f1-score   support

   high risk       0.85      0.84      0.84        94
    low risk       0.69      0.81      0.74       139
    mid risk       0.56      0.44      0.49       102

    accuracy                           0.70       335
   macro avg       0.70      0.70      0.69       335
weighted avg       0.70      0.70      0.70       335



In [39]:
#modelo2: 1 capa de entrada, 1 capa oculta de 100 neuronas, 1 capa de salida de 3 neuronas:
model2 = Sequential()
model2.add(Dense(100, input_dim=6, activation='relu'))
model2.add(Dense(3, activation='sigmoid'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [40]:
model2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 100)               700       
                                                                 
 dense_11 (Dense)            (None, 3)                 303       
                                                                 
Total params: 1,003
Trainable params: 1,003
Non-trainable params: 0
_________________________________________________________________


In [ ]:
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=1000)
mc = ModelCheckpoint('best_model2.h5', monitor='accuracy', mode='max', verbose=1, save_best_only=True)
model2.fit(X_train, dummy_ytrain, epochs=1000, batch_size=16, verbose=2, callbacks=[es,mc])


In [42]:
modelo2 = load_model('best_model2.h5')
ypred3 = modelo2.predict(X_test)
#ypred3 = ypred3.round()
ypred3 = np.argmax(ypred3, axis=1)
ypred3 = encoder.inverse_transform(ypred3)


11/11 [==============================] - 0s 2ms/step


In [43]:
cm4 = confusion_matrix(y_test, ypred3)
print(f'CM:',cm4)
print(f'Accuracy:',accuracy_score(y_test, ypred3)* 100 ,'%')
print(classification_report(y_test, ypred3))



CM: [[ 78   2  14]
 [  5 100  34]
 [  9  34  59]]
Accuracy: 70.74626865671641 %
              precision    recall  f1-score   support

   high risk       0.85      0.83      0.84        94
    low risk       0.74      0.72      0.73       139
    mid risk       0.55      0.58      0.56       102

    accuracy                           0.71       335
   macro avg       0.71      0.71      0.71       335
weighted avg       0.71      0.71      0.71       335



In [44]:
#modelo3: 1 capa de entrada, 2 capas ocultas de 100 neuronas, 1 capa de salida de 3 neuronas:
model3 = Sequential()
model3.add(Dense(100, input_dim=6, activation='relu'))
model3.add(Dense(100, activation='sigmoid'))
model3.add(Dense(3, activation='sigmoid'))
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [45]:
model3.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 100)               700       
                                                                 
 dense_13 (Dense)            (None, 100)               10100     
                                                                 
 dense_14 (Dense)            (None, 3)                 303       
                                                                 
Total params: 11,103
Trainable params: 11,103
Non-trainable params: 0
_________________________________________________________________


In [ ]:
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=1000)
mc = ModelCheckpoint('best_model3.h5', monitor='accuracy', mode='max', verbose=1, save_best_only=True)
model3.fit(X_train, dummy_ytrain, epochs=1000, batch_size=16, verbose=2, callbacks=[es,mc])


In [47]:
modelo3 = load_model('best_model3.h5')
ypred3 = modelo3.predict(X_test)
#ypred3 = ypred3.round()
ypred3 = np.argmax(ypred3, axis=1)
ypred3 = encoder.inverse_transform(ypred3)
#ypred3


11/11 [==============================] - 0s 2ms/step


In [48]:
cm5 = confusion_matrix(y_test, ypred3)
print(f'CM:',cm5)
print(f'Accuracy:',accuracy_score(y_test, ypred3)* 100 ,'%')
print(classification_report(y_test, ypred3))


CM: [[ 75   6  13]
 [  4 127   8]
 [  5  56  41]]
Accuracy: 72.53731343283583 %
              precision    recall  f1-score   support

   high risk       0.89      0.80      0.84        94
    low risk       0.67      0.91      0.77       139
    mid risk       0.66      0.40      0.50       102

    accuracy                           0.73       335
   macro avg       0.74      0.70      0.71       335
weighted avg       0.73      0.73      0.71       335



In [49]:
#modelo4: 1 capa de entrada, 3 capas ocultas de 100 neuronas, 1 capa de salida de 3 neuronas:
model4 = Sequential()
model4.add(Dense(100, input_dim=6, activation='relu'))
model4.add(Dense(100, activation='sigmoid'))
model4.add(Dense(100, activation='sigmoid'))
model4.add(Dense(3, activation='sigmoid'))
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [50]:
model4.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 100)               700       
                                                                 
 dense_16 (Dense)            (None, 100)               10100     
                                                                 
 dense_17 (Dense)            (None, 100)               10100     
                                                                 
 dense_18 (Dense)            (None, 3)                 303       
                                                                 
Total params: 21,203
Trainable params: 21,203
Non-trainable params: 0
_________________________________________________________________


In [ ]:
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=1000)
mc = ModelCheckpoint('best_model4.h5', monitor='accuracy', mode='max', verbose=1, save_best_only=True)
model4.fit(X_train, dummy_ytrain, epochs=1000, batch_size=16, verbose=2, callbacks=[es,mc])


In [52]:
modelo4 = load_model('best_model4.h5')
ypred3 = modelo4.predict(X_test)
#ypred3 = ypred3.round()
ypred3 = np.argmax(ypred3, axis=1)
ypred3 = encoder.inverse_transform(ypred3)
#ypred3

11/11 [==============================] - 0s 2ms/step


In [53]:
cm6 = confusion_matrix(y_test, ypred3)
print(f'CM:',cm6)
print(f'Accuracy:',accuracy_score(y_test, ypred3)* 100 ,'%')
print(classification_report(y_test, ypred3))


CM: [[ 77   2  15]
 [  3 114  22]
 [  9  40  53]]
Accuracy: 72.83582089552239 %
              precision    recall  f1-score   support

   high risk       0.87      0.82      0.84        94
    low risk       0.73      0.82      0.77       139
    mid risk       0.59      0.52      0.55       102

    accuracy                           0.73       335
   macro avg       0.73      0.72      0.72       335
weighted avg       0.73      0.73      0.72       335



In [56]:
#modelo5: 1 capa de entrada, 3 capas ocultas de 500 neuronas, 1 capa de salida de 3 neuronas:
model5 = Sequential()
model5.add(Dense(500, input_dim=6, activation='relu'))
model5.add(Dense(500, activation='sigmoid'))
model5.add(Dense(500, activation='sigmoid'))
model5.add(Dense(3, activation='sigmoid'))
model5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [57]:
model5.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 500)               3500      
                                                                 
 dense_20 (Dense)            (None, 500)               250500    
                                                                 
 dense_21 (Dense)            (None, 500)               250500    
                                                                 
 dense_22 (Dense)            (None, 3)                 1503      
                                                                 
Total params: 506,003
Trainable params: 506,003
Non-trainable params: 0
_________________________________________________________________


In [ ]:
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=1000)
mc = ModelCheckpoint('best_model5.h5', monitor='accuracy', mode='max', verbose=1, save_best_only=True)
model5.fit(X_train, dummy_ytrain, epochs=1000, batch_size=16, verbose=2, callbacks=[es,mc])


In [59]:
modelo5 = load_model('best_model5.h5')
ypred3 = modelo5.predict(X_test)
#ypred3 = ypred3.round()
ypred3 = np.argmax(ypred3, axis=1)
ypred3 = encoder.inverse_transform(ypred3)
#ypred3

11/11 [==============================] - 0s 4ms/step


In [60]:
cm7 = confusion_matrix(y_test, ypred3)
print(f'CM:',cm7)
print(f'Accuracy:',accuracy_score(y_test, ypred3)* 100 ,'%')
print(classification_report(y_test, ypred3))

CM: [[ 81   0  13]
 [  6 102  31]
 [  6  21  75]]
Accuracy: 77.01492537313432 %
              precision    recall  f1-score   support

   high risk       0.87      0.86      0.87        94
    low risk       0.83      0.73      0.78       139
    mid risk       0.63      0.74      0.68       102

    accuracy                           0.77       335
   macro avg       0.78      0.78      0.77       335
weighted avg       0.78      0.77      0.77       335



* Tabla comparativa:





In [61]:

from tabulate import tabulate
data = [["Árbol de Clasif.","--",0.69,0.86,0.74,0.90,0.79],
["Random Forest","--",0.70,0.86,0.76,0.88,0.81],
["RN1 (1,50)",503,0.56,0.85,0.44,0.84,0.70],
["RN2 (1,100)",1003,0.55,0.85,0.58,0.83,0.71],
["RN3 (2,100)",11103,0.66,0.89,0.40,0.91,0.73],
["RN4 (3,100)",21203,0.59,0.87,0.52,0.82,0.73],
["RN5 (3,500)",506003,0.63,0.87,0.73,0.86,0.77]
]
print (tabulate(data, headers=["Método", "Parámetros", "Min Precision", "Max Precision", "Min Recall", "Max Recall", "Accuracy (global)"]))

Método            Parámetros      Min Precision    Max Precision    Min Recall    Max Recall    Accuracy (global)
----------------  ------------  ---------------  ---------------  ------------  ------------  -------------------
Árbol de Clasif.  --                       0.69             0.86          0.74          0.9                  0.79
Random Forest     --                       0.7              0.86          0.76          0.88                 0.81
RN1 (1,50)        503                      0.56             0.85          0.44          0.84                 0.7
RN2 (1,100)       1003                     0.55             0.85          0.58          0.83                 0.71
RN3 (2,100)       11103                    0.66             0.89          0.4           0.91                 0.73
RN4 (3,100)       21203                    0.59             0.87          0.52          0.82                 0.73
RN5 (3,500)       506003                   0.63             0.87          0.73          0